In [ ]:
import mlrose_hiive as mr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time

In [ ]:
# Define the K-Color optimization problem
problem_size = 100 
edges = [(0, 1), (0, 2), (0, 4), (1, 3), (2, 0), (2, 3), (3, 4)]
fitness_kcolor = mr.MaxKColor(edges)
problem_kcolor = mr.DiscreteOpt(length=problem_size, fitness_fn=fitness_kcolor, maximize=True, max_val=5)

In [ ]:
max_attempts = 200
max_iterations = 1000  # Limit iterations for quicker execution
num_runs = 10  # Number of runs for statistical robustness

In [ ]:
# Initialize results storage
results = {}

# List of algorithms and their specific parameters
algorithms = {
    'random_hill_climb': {'restarts': 50},
    'simulated_annealing': {'schedule': mr.ExpDecay(init_temp=1, exp_const=0.005, min_temp=0.001)},
    'genetic_alg': {'pop_size': 200, 'mutation_prob': 0.1}
}

In [ ]:
# Execute each algorithm multiple times
for name, params in algorithms.items():
    run_times = []
    fitness_scores = []
    curves = []
    total_fevs = []  # List to store total function evaluations for each run

    for _ in range(num_runs):
        start_time = time.time()
        _, best_fitness, fitness_curve = getattr(mr, name.lower())(
            problem_kcolor, max_attempts=max_attempts, max_iters=max_iterations, 
            curve=True, random_state=42, **params)
        end_time = time.time()

        run_times.append(end_time - start_time)
        fitness_scores.append(best_fitness)
        curves.append(fitness_curve[:, 0])  # Assuming the first column is the fitness score

        # Assuming the second column of fitness_curve contains the cumulative fevals
        total_fevs.append(fitness_curve[-1, 1])

    # Calculate fevals per second and per iteration
    fevals_per_sec = sum(total_fevs) / sum(run_times)
    fevals_per_iteration = sum(total_fevs) / (num_runs * max_iterations)

    results[name] = {
        'times': run_times,
        'fitness_scores': fitness_scores,
        'average_curve': np.mean(curves, axis=0),
        'total_fevs': sum(total_fevs),  # Total function evaluations across all runs
        'fevals_per_sec': fevals_per_sec,  # Function evaluations per second
        'fevals_per_iteration': fevals_per_iteration  # Function evaluations per iteration
    }


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv('kcolor_results.csv')
results_df

In [ ]:
import matplotlib.pyplot as plt

# Plot average fitness curves
plt.figure(figsize=(10, 6))
for algo, data in results.items():
    plt.plot(data['average_curve'], label=f"{algo} (Avg. Fitness)")
plt.title('Average Fitness Curves for 4-Peaks Problem')
plt.xlabel('Iterations')
plt.ylabel('Fitness')
plt.legend()
plt.grid(True)
plt.show()

# Plot convergence times
plt.figure(figsize=(10, 6))
convergence_times = [np.mean(data['times']) for algo, data in results.items()]
plt.bar(results.keys(), convergence_times, color='lightblue')
plt.title('Average Convergence Times for 4-Peaks Problem')
plt.xlabel('Algorithm')
plt.ylabel('Time in seconds')
plt.grid(True, axis='y')
plt.show()
